In [537]:
import problema_espacio_estados as probee
import búsqueda_espacio_estados as búsqee
import copy

In [538]:
class Hitori:
    def __init__(self, casillas):
        self.casillas = casillas
    
    def tamaño_hor(self):
        return len(self.casillas[0])
    
    def tamaño_ver(self):
        return len(self.casillas)
    
    def valor_casilla(self, f, c):
        return self.casillas[f][c]    
    
    def __str__(self):
        cadena = str(self.casillas)
        
        return cadena
    
    def es_aplicable(self,estado):
        result = True
        
        for i in range(0, Hitori.tamaño_hor(estado)-1):
            for j in range(0, Hitori.tamaño_ver(estado)-1):
                
                if Hitori.valor_casilla(estado,i,j) <= max(Hitori.tamaño_hor(estado), Hitori.tamaño_ver(estado)):
                    result = result and True
                else:
                    result = result and False

In [539]:
class Poner_negro(probee.Acción):
    def __init__(self, i, j):
        nombre = 'posición {},{}'.format(i, j)
        super().__init__(nombre)
        self.f = i
        self.c = j
        
    def es_negro(self, estado, f, c):
        return bool(Hitori.valor_casilla(estado,f,c) == 0)
    
    def es_blanco(self, f, c):
        return not bool(Hitori.valor_casilla(self,f,c) == 0)
    
    def negro_colindante(self, estado):
        arriba = False
        abajo = False
        izquierda = False
        derecha = False
        
        if self.f < (Hitori.tamaño_ver(estado)-1):
            abajo = self.es_negro(estado, (self.f)+1, self.c)
         
        if self.f > 0:
            arriba = self.es_negro(estado, (self.f)-1, self.c)
            
        if self.c < (Hitori.tamaño_hor(estado)-1):
            derecha = self.es_negro(estado, self.f, (self.c)+1)
            
        if self.c > 0:
            izquierda = self.es_negro(estado, self.f, (self.c)-1)
            
        return arriba or abajo or izquierda or derecha
    
    def duplicado_fila(self, estado):
        result = False
        tamaño = Hitori.tamaño_hor(estado)
        limite_izq = self.c
        limite_der = tamaño
        for i in range(0, limite_izq):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                if Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c):
                    result = True
                
        for j in range((self.c)+1, limite_der):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                 if Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c):
                    result = True
                
        return result
    
    def duplicado_columna(self, estado):
        result = False
        tamaño = Hitori.tamaño_ver(estado)
        limite_izq = self.f
        limite_der = tamaño
        for i in range(0, limite_izq):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                if Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    result = True
                
        for j in range((self.f)+1, limite_der):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                 if Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    result = True
                
        return result
    
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |     |  0     |
    #          -----------------------
    #          |       |  x  |        |
    #          -----------------------
    #
    #
    #
    #
    def abajo_encierra(self, estado):
        arriba = True
        arriba_derecha = True
        arriba_izquierda = True
        
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba_izquierda = False        
                
        if self.f > 0 and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba_derecha = False
                
        if self.f > 1: 
            if not self.es_negro(estado, (self.f)-2, self.c):
                arriba = False
                
        if self.f == 0:
            arriba = False
            arriba_derecha = False
            arriba_izquierda = False
            
            
        return arriba and arriba_derecha and arriba_izquierda    
        
    #
    #
    #           ------------------------
    #          |       |  x  |        |
    #          ------------------------
    #          |    0  |     |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def arriba_encierra(self, estado):
        abajo = True
        abajo_derecha = True
        abajo_izquierda = True
        
        if self.f < Hitori.tamaño_ver(estado)-1 and self.c > 0:
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo_izquierda = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo_derecha = False
                
        if self.f < (Hitori.tamaño_ver(estado)-2): 
            if not self.es_negro(estado, (self.f)+2, self.c):
                abajo = False
                
        if self.f == (Hitori.tamaño_ver(estado)-1):
            abajo = False
            abajo_derecha = False
            abajo_izquierda = False
            
            
        return abajo and abajo_derecha and abajo_izquierda    
      
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |     |  x     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def derecha_encierra(self, estado):
        izquierda = True
        arriba = True
        abajo = True
        
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c > 0:      
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo = False
                
        if self.c > 1: 
            if not self.es_negro(estado, (self.f), self.c-2):
                izquierda = False
                
        if self.c == 0:
            izquierda = False
            arriba = False
            abajo = False
            
            
        return izquierda and arriba and abajo  
    
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    x  |     |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def izquierda_encierra(self, estado):
        derecha = True
        arriba = True
        abajo = True
        
        if self.f > 0 and self.c < Hitori.tamaño_hor(estado)-1:
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo = False
                
        if self.c < Hitori.tamaño_hor(estado)-2: 
            if not self.es_negro(estado, (self.f), self.c+2):
                derecha = False
                
        if self.c == (Hitori.tamaño_hor(estado)-1):
            derecha = False
            arriba = False
            abajo = False
            
            
        return derecha and arriba and abajo    
     
        
    #Si se cumple alguna de las restricciones, poner negro la casilla provoca de se encierre una casilla
    def encierra(self, estado):
        abajo_encierra = self.abajo_encierra(estado)
        arriba_encierra = self.arriba_encierra(estado)
        izquierda_encierra = self.izquierda_encierra(estado)
        derecha_encierra = self.derecha_encierra(estado)
        
        
        return abajo_encierra or arriba_encierra or izquierda_encierra or derecha_encierra
    
    def diagonal_arriba_derecha(self, estado):
        limite = min(self.f, Hitori.tamaño_hor(estado)-1-self.c)
        result = False
        if self.f != 0 and (self.c != Hitori.tamaño_hor(estado)-1):
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f-ac, self.c+ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result
    
    def diagonal_arriba_izquierda(self, estado):
        limite = min(self.f, self.c)
        result = False
        if self.f != 0 and self.c != 0:
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f-ac, self.c-ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result   
    
    def diagonal_abajo_derecha(self, estado):
        limite = min(Hitori.tamaño_ver(estado)-1-self.f, Hitori.tamaño_hor(estado)-1-self.c)
        result = False
        if (self.f != Hitori.tamaño_ver(estado)-1) and (self.c != Hitori.tamaño_hor(estado)-1):
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f+ac, self.c+ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result
      
    def diagonal_abajo_izquierda(self, estado):
        limite = min(Hitori.tamaño_ver(estado)-1-self.f, self.c)
        result = False
        if (self.f != Hitori.tamaño_ver(estado)-1) and self.c != 0:
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f+ac, self.c-ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result 
    
    def parte_matriz(self, estado):
        arriba_derecha = self.diagonal_abajo_derecha
        arriba_izquierda = self.diagonal_abajo_izquierda
        abajo_derecha = self.diagonal_abajo_derecha
        abajo_izquierda = self.diagonal_abajo_izquierda
        
        return ((arriba_derecha and arriba_izquierda) 
                or (arriba_derecha and abajo_derecha) 
                or (arriba_derecha and abajo_izquierda) 
                or (arriba_izquierda and arriba_derecha) 
                or (arriba_izquierda and abajo_derecha) 
                or (abajo_derecha and abajo_izquierda))
        
    def es_aplicable(self, estado):
        return bool(not self.negro_colindante(estado)
                    and not self.parte_matriz(estado)
                    and not self.encierra(estado)
                    and (self.duplicado_fila(estado)
                        or self.duplicado_columna(estado)))
        
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        nuevo_estado.casillas[self.f][self.c] = 0
        return nuevo_estado
    
    def es_estado_final(self, estado):
        return not bool(self.duplicado_fila(estado) or self.duplicado_columna(estado))

In [540]:
acciones[7].arriba_encierra(estado_inicial)

False

In [541]:
estado_inicial = Hitori([[1,2,1],
                         [2,2,1],
                         [3,1,2]])

In [542]:
estado_final = Hitori([[1,2,0],
                       [2,0,1],
                       [3,1,2]])

In [543]:
print(Hitori.tamaño_hor(estado_inicial))

3


In [544]:
acciones = [Poner_negro (i,j) for i in range(0, Hitori.tamaño_hor(estado_inicial)) for j in range(0, Hitori.tamaño_ver(estado_inicial))]

In [545]:
acciones

In [546]:
Hitori.valor_casilla(estado_inicial,0,1)


2

In [547]:
problema = probee.ProblemaEspacioEstados(acciones, estado_inicial, [])


In [548]:
print("b_optima_nueva.buscar[problema]")
b_optima_nueva = búsqee.BúsquedaEnProfundidad(detallado=True)
print("\n")
print("b_optima_nueva.buscar[problema]")
b_optima_nueva.buscar(problema)

b_optima_nueva.buscar[problema]


b_optima_nueva.buscar[problema]
Nodo: Estado: [[1, 2, 1], [2, 2, 1], [3, 1, 2]]; Prof: 0


In [549]:
acciones[4].es_aplicable(estado_inicial)

False

In [550]:
acciones[8].diagonal_arriba_izquierda(estado_inicial)

False

In [551]:
print(estado_inicial)

[[1, 2, 1], [2, 2, 1], [3, 1, 2]]


In [552]:
acciones[5].es_estado_final(estado_inicial)

False